In [71]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from pythermalcomfort.models import pmv
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv("lstm_clean.csv")
df

,ITEMP,IAV,IRH,IMRT,CCSP,CHSP,OTEMP,ORH,OAV,YEAR,MONTH,DAY,PMV
0,24.584695,63.517,0.030988,24.584695,18.333333,26.666667,23.0,89,1.34112,2012,7,4,-0.270899
1,24.584695,63.704,0.030988,24.584695,18.333333,26.666667,23.0,89,1.34112,2012,7,4,-0.269505
2,24.584695,63.856,0.030988,24.584695,18.333333,26.666667,23.0,89,1.34112,2012,7,4,-0.268373
3,24.584695,64.039,0.030988,24.584695,18.333333,26.666667,23.0,89,0.89408,2012,7,4,-0.267009
4,24.498384,64.282,0.030988,24.498384,18.333333,26.666667,23.0,89,0.89408,2012,7,4,-0.297025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619757,23.593889,55.626,0.030988,23.593889,18.333333,26.666667,17.0,72,1.34112,2012,9,4,-0.537396
619758,23.775000,55.626,0.030988,23.775000,18.333333,26.666667,17.0,72,1.34112,2012,9,4,-0.474448
619759,23.775000,55.626,0.030988,23.775000,18.333333,26.666667,17.0,72,1.34112,2012,9,4,-0.474448
619760,23.775000,55.626,0.030988,23.775000,18.333333,26.666667,16.0,77,1.34112,2012,9,4,-0.474448


In [18]:
X_all = df[['CCSP','CHSP','OTEMP','ORH','OAV','YEAR','MONTH','DAY']]
y_all = df[['ITEMP','IRH','IMRT','PMV']]
split_ = int(len(X_all)*0.8)
x_train_all, x_test_all = X_all[:split_], X_all[split_:]
y_train_all, y_test_all = y_all[:split_], y_all[split_:]

In [19]:
scaler_all = StandardScaler()
x_train_all = scaler_all.fit_transform(x_train_all)
x_test_all = scaler_all.transform(x_test_all)

In [21]:
def create_dataset(dataset, time_step=1):
    dataX = []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step)]
        dataX.append(a)
#         dataY.append(dataset[i+time_step])
    return np.array(dataX), np.array(dataY)

In [78]:
time_step = 100
x_train, y_train = create_dataset(x_train_all,time_step)
x_test, y_test = create_dataset(x_test_all,time_step)

In [79]:
x_train.shape

(495708, 100, 8)

In [80]:
model_all = Sequential()
model_all.add(LSTM(50,return_sequences = True,input_shape=(100,8)))
# model_all.add(LSTM(50,return_sequences = True))
# model_all.add(LSTM(50))
# model_all.add(Dense(8,activation='relu'))
# model.output_shape
model_all.add(Flatten())
model_all.add(Dense(3,activation = 'linear'))
model_all.compile(loss='mean_squared_error',optimizer='adam')

In [81]:
model_all.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 100, 50)           11800     
                                                                 
 flatten_1 (Flatten)         (None, 5000)              0         
                                                                 
 dense_15 (Dense)            (None, 3)                 15003     
                                                                 
Total params: 26,803
Trainable params: 26,803
Non-trainable params: 0
_________________________________________________________________


In [82]:
y_train_final = y_train_all.drop(['PMV'],axis=1)
y_train_final[:495708].shape,x_train.shape

((495708, 3), (495708, 100, 8))

In [83]:
history_all = model_all.fit(x_train,y_train_final[:495708],validation_split=0.2,epochs=10)

Epoch 1/10
12393/12393 [==============================] - 1693s 136ms/step - loss: 1.4726 - val_loss: 1.5333
Epoch 2/10
12393/12393 [==============================] - 1594s 129ms/step - loss: 0.7237 - val_loss: 1.3699
Epoch 3/10
12393/12393 [==============================] - 1355s 109ms/step - loss: 0.6186 - val_loss: 1.5678
Epoch 4/10
12393/12393 [==============================] - 799s 64ms/step - loss: 0.5406 - val_loss: 1.4208
Epoch 5/10
12393/12393 [==============================] - 797s 64ms/step - loss: 0.4971 - val_loss: 1.4207
Epoch 6/10
12393/12393 [==============================] - 802s 65ms/step - loss: 0.4465 - val_loss: 1.4880
Epoch 7/10
12393/12393 [==============================] - 810s 65ms/step - loss: 0.4134 - val_loss: 1.3607
Epoch 8/10
12393/12393 [==============================] - 805s 65ms/step - loss: 0.3855 - val_loss: 1.4814
Epoch 9/10
12393/12393 [==============================] - 813s 66ms/step - loss: 0.3578 - val_loss: 1.5445
Epoch 10/10
12393/12393 [======

In [ ]:
# ((495708, 100, 8), (495809, 4))